In [1]:
import psycopg2

import random
import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

import joblib

import torch
from torch import nn
from torch.utils.data import Subset, DataLoader

from Dataset.Embedding_Dataset import Embedding_Dataset
from Model.Embedding import Embedding

from Dataset.Apartment_Complex_Dataset import Apartment_Complex_Dataset
from Model.LSTM import LSTM
from Model.GRU import GRU
from Model.Transformer import Transformer

from Dataset.Dong_Dataset import Dong_Dataset
from Model.LSTM_Attention import LSTMAttention
from Model.GRU_Attention import GRUAttention
from Model.Transformer_Attention import TransformerAttention

from utils import *
from sklearn.metrics import mean_squared_error
def MAPE(y_test, y_pred):
    return torch.mean(torch.abs((y_test - y_pred) / y_test)) * 100


SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DEVICE = torch.device('cpu')

## Hyperparameters

In [2]:
embedding_dim = 1024
window_size = 10
batch_size = 1

ml_batch = 128
ml_estimators = 150
ml_window_size = 10

## Data

In [3]:
table_1 = pd.read_csv('../데이터/Table/test_table_1.csv') 
table_2 = pd.read_csv('../데이터/Table/test_table_2.csv') 
table_3 = pd.read_csv('../데이터/Table/test_table_3.csv') 

In [4]:
table_merge = pd.merge(table_1, table_3, how='left', on='aid')
table_merge = pd.merge(table_merge, table_2, how='left', on='did')
len(table_merge)

1874

## DL Dataset & Dataloader

In [22]:
# concat_12_dim
# dataset = Dong_Dataset('None', table_1, table_2, table_3, 'None', window_size, 'TEST', DEVICE)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)

# emb_1024_dim
model = torch.load('../데이터/Checkpoint/embedding_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_6_e1_128_e2_128_e3_512_emb_1024_d1_512_d2_256_d3_128.pth', map_location=torch.device('cpu'))
dataset = Dong_Dataset(model, table_1, table_2, table_3, embedding_dim, window_size, 'TEST', DEVICE)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, drop_last=True)

In [25]:
# LSTM
# model = torch.load("../데이터/Checkpoint/concat_12_dim/lstm_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_124_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/lstm_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_64_hdim_256_ws_3.pth", map_location=torch.device('cpu'))

# GRU
# model = torch.load("../데이터/Checkpoint/concat_12_dim/gru_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_321_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/gru_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_71_hdim_256_ws_3.pth", map_location=torch.device('cpu'))

# nlinear
# model = torch.load("../데이터/Checkpoint/concat_12_dim/nlinear_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_118_emb_12_ws_10.pth", map_location=torch.device('cpu'))

# transformer
# model = torch.load("../데이터/Checkpoint/concat_12_dim/transformer_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_63_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/transformer_tr_0.8_lr_0.0001_wd_0_batch_128_epochs_5_ws_3.pth", map_location=torch.device('cpu'))

# LSTM attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/lstm_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/dong/1024_dim/10_ws/lstm_att_tr_0.8_lr_0.0001_wd_0_batch_1_epochs_34_hdim_256_ws_10.pth", map_location=torch.device('cpu'))

# GRU attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/gru_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/gru_att_tr_0.8_lr_1e-06_wd_0_batch_1_epochs_4_hdim_256_ws_10.pth", map_location=torch.device('cpu'))

# NLinear attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/nlinear_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_6.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/emb_1024_dim/", map_location=torch.device('cpu'))

# Transformer attention
# model = torch.load("../데이터/Checkpoint/concat_12_dim/transformer_att_tr_0.8_lr_1e-05_wd_0_batch_1_epochs_7_hdim_12_ws_10.pth", map_location=torch.device('cpu'))
# model = torch.load("../데이터/Checkpoint/dong/1024_dim/10_ws/transformer_att_tr_0.8_lr_0.0001_wd_0_batch_1_epochs_11_hdim_1024_ws_10.pth", map_location=torch.device('cpu'))

In [26]:
count = 0
RMSE_total_loss = 0
MSE_total_loss = 0
MAPE_total_loss = 0

with torch.no_grad():
    for i, batch in enumerate(dataloader):
        src = batch[0][0].to(DEVICE)
        max_len = batch[1][0].to(DEVICE)
        anw = batch[2][0]
        trg = batch[3][0].to(DEVICE)
        
        if len(anw) == 0:
            continue
        
        RMSE_epoch_loss = 0
        MSE_epoch_loss = 0
        MAPE_epoch_loss = 0
        count += anw.shape[0]

        for index in anw:
            index.to(DEVICE)
            # LSTM
            # output, _, _ = model(src)
            
            # GRU
            # output, _ = model(src)
            
            # nlinear
            # output, _ = model(src)
            
            # transformer
            # src_mask = model.generate_square_subsequent_mask(src.shape[1]).to(src.device)
            # output, _ = model(src, src_mask)
            
            # RMSE_loss = np.sqrt(mean_squared_error(output[index], trg[index]))
            # MSE_loss = mean_squared_error(output[index], trg[index])
            # MAPE_loss = MAPE(output[index], trg[index])
            
            # attention 계열
            output = model(src, index, max_len)

            RMSE_loss = np.sqrt(mean_squared_error(output, trg[index]))
            MSE_loss = mean_squared_error(output, trg[index])
            MAPE_loss = MAPE(output, trg[index])
            
        RMSE_total_loss += RMSE_loss
        MSE_total_loss += MSE_loss
        MAPE_total_loss += MAPE_loss

    print(f'Test RMSE Loss: {RMSE_total_loss/count:.4f}')
    print(f'Test MSE Loss: {MSE_total_loss/count:.4f}')
    print(f'Test MAPE Loss: {MAPE_total_loss/count:.4f}')

Test RMSE Loss: 1.8382
Test MSE Loss: 15.9428
Test MAPE Loss: 23.2572


## ML Dataset & Dataloader

In [ ]:
# concat_12_dim
dataset = Apartment_Complex_Dataset('None', table_1, table_2, table_3, 'None', ml_window_size, 'ML', 'TEST', DEVICE)
dataloader = DataLoader(dataset, batch_size=ml_batch, shuffle=False, drop_last=True)

# emb_1024_dim
# model = torch.load('../데이터/Checkpoint/embedding_tr_0.8_lr_1e-05_wd_0_batch_128_epochs_131_e1_128_e2_128_e3_512_emb_1024_d1_512_d2_256_d3_128.pth', map_location=torch.device('cpu'))
# dataset = Apartment_Complex_Dataset(model, table_1, table_2, table_3, embedding_dim, ml_window_size, 'ML', 'TEST', DEVICE)
# dataloader = DataLoader(dataset, batch_size=ml_batch, shuffle=False, drop_last=True)

In [ ]:
RMSE_criterion = np.sqrt(mean_squared_error())
MSE_criterion = mean_squared_error()
MAPE_criterion = MAPE()

RMSE_total_loss = 0
MSE_total_loss = 0
MAE_total_loss = 0
MAPE_total_loss = 0

for i, data in enumerate(dataloader):
    X, y = data[0].squeeze().cpu().numpy(), data[1].squeeze().cpu().numpy()
    y_pred = model.predict(X)
    RMSE_loss = RMSE_criterion(y, y_pred)
    RMSE_total_loss += RMSE_loss/len(dataloader)
    MSE_loss = MSE_criterion(y, y_pred)
    MSE_total_loss += MSE_loss/len(dataloader)
    MAPE_loss = MAPE_criterion(y, y_pred)
    MAPE_total_loss += MAPE_loss/len(dataloader)

print(f'Test RMSE Loss: {RMSE_total_loss:.4f}')
print(f'Test MSE Loss: {MSE_total_loss:.4f}')
print(f'Test MAE Loss: {MAE_total_loss:.4f}')
print(f'Test MAPE Loss: {MAPE_total_loss:.4f}')